In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

urls = {
    "2025/26": "https://resultater.volleyball.dk/tms/Turneringer-og-resultater/Pulje-Komplet-Kampprogram.aspx?PuljeId=3752",
    "2024/25": "https://resultater.volleyball.dk/tms/Turneringer-og-resultater/Pulje-Komplet-Kampprogram.aspx?PuljeId=3925"
}

team = "Frederiksberg Volley"
all_matches = []

for season, url in urls.items():
    resp = requests.get(url)
    resp.raise_for_status()
    soup = BeautifulSoup(resp.text, "html.parser")

    # Find the schedule table
    table = soup.find("table", {"class": "srDefault srProgramNormal"})
    for row in table.find_all("tr"):
        cols = row.find_all("td")
        if len(cols) < 6:
            continue

        kampnr = cols[0].get_text(strip=True)
        date_time = cols[1].get_text(" ", strip=True)
        hjemme = cols[2].get_text(strip=True)
        ude = cols[3].get_text(strip=True)
        venue = cols[4].get_text(" ", strip=True)
        result = cols[5].get_text(strip=True)

        if team in hjemme or team in ude:
            match = {
                #"season": season,
                "kampnr": kampnr,
                "dato_tid": date_time,
                "hjemme": hjemme,
                "ude": ude,
                "spillested": venue,
                "resultat": result,
            }
            all_matches.append(match)

df = pd.DataFrame(all_matches)

# --- Add won/lost column ---
def team_result(row):
    try:
        home_sets, away_sets = [int(x) for x in row["resultat"].replace("\xa0", " ").split("-")]
    except Exception:
        return None  # no result yet

    if team == row["hjemme"] and home_sets > away_sets:
        return "won"
    elif team == row["ude"] and away_sets > home_sets:
        return "won"
    else:
        return "lost"

df["result_for_frb"] = df.apply(team_result, axis=1)

# Extract only the date part
df["date"] = df["dato_tid"].str.split(" kl.").str[0]

# Convert to datetime and format as YYYY-MM-DD
df["date"] = pd.to_datetime(df["date"], format="%d-%m-%y").dt.strftime("%Y-%m-%d")

team_abbr = {
    "Frederiksberg Volley": "frb",
    "Team Køge": "tkv",
    "Amager Volley.2": "avk",
    "Lyngby-Gladsaxe VK": "lgv",
    "Hvidovre VK.2": "hv2",
    "Gentofte Volley.2": "gv2",
    "Farum-Holte": "fah"
    # Add more teams as needed
}



df["match_name"] = df.apply(
    lambda row: f"{team_abbr.get(row['hjemme'], row['hjemme'][:3].lower())}-"
                f"{team_abbr.get(row['ude'], row['ude'][:3].lower())}",
    axis=1
)

# Make sure team names are consistent
frb_name = "Frederiksberg Volley"

# Create opponent column
df["opponent"] = df.apply(
    lambda row: row["ude"] if row["hjemme"] == frb_name else row["hjemme"],
    axis=1
)

df["played"] = df["resultat"].str.strip().apply(lambda x: "yes" if x != "" else "no")


# Add season based on match date
def assign_season(date):
    year = date.year
    month = date.month
    if month >= 9:
        return f"{year}/{year + 1}"
    else:
        return f"{year - 1}/{year}"

df["date"] = pd.to_datetime(df["date"], errors='coerce')
df["season"] = df["date"].apply(assign_season)

df.to_csv("match-results-dvbf.csv", index=False)
df

,kampnr,dato_tid,hjemme,ude,spillested,resultat,result_for_frb,date,match_name,opponent,played,season
0,141063,29-09-24 kl. 10:00,Gentofte Volley.2,Frederiksberg Volley,Kildeskovshal 1 1,3 - 2,lost,2024-09-29,gv2-frb,Gentofte Volley.2,yes,2024/2025
1,141067,06-10-24 kl. 18:00,Frederiksberg Volley,KV 61,Bülowsvejhallen 1,3 - 0,won,2024-10-06,frb-kv,KV 61,yes,2024/2025
2,141074,13-10-24 kl. 14:00,Frederiksberg Volley,Amager Volley.2,Kedelhallen 1,0 - 3,lost,2024-10-13,frb-avk,Amager Volley.2,yes,2024/2025
3,141078,27-10-24 kl. 11:30,VLI,Frederiksberg Volley,Kedelhallen 1,3 - 0,lost,2024-10-27,vli-frb,VLI,yes,2024/2025
4,141089,02-11-24 kl. 18:00,Frederiksberg Volley,Team Køge,Bülowsvejhallen 1,3 - 0,won,2024-11-02,frb-tkv,Team Køge,yes,2024/2025
5,141094,16-11-24 kl. 14:30,DTU Volley,Frederiksberg Volley,Engelsborghallen 1,2 - 3,won,2024-11-16,dtu-frb,DTU Volley,yes,2024/2025
6,141099,23-11-24 kl. 14:00,Frederiksberg Volley,Hvidovre VK.2,Bülowsvejhallen 1,2 - 3,lost,2024-11-23,frb-hv2,Hvidovre VK.2,yes,2024/2025
7,141104,01-12-24 kl. 14:00,Grøndal EV,Frederiksberg Volley,Hillerødgades Sportssal 1,0 - 3,won,2024-12-01,grø-frb,Grøndal EV,yes,2024/2025
8,141108,08-12-24 kl. 18:00,Frederiksberg Volley,Gentofte Volley.2,Bülowsvejhallen 1,3 - 1,won,2024-12-08,frb-gv2,Gentofte Volley.2,yes,2024/2025
9,141119,12-01-25 kl. 12:00,Amager Volley.2,Frederiksberg Volley,Sundbyhal 1 1,3 - 2,lost,2025-01-12,avk-frb,Amager Volley.2,yes,2024/2025
